In [72]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [73]:
dataset = pd.read_csv("credit_simple.csv", sep=";")
dataset.shape

(1000, 8)

In [74]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [75]:
# separando variável dependente das variáveis independentes
y = dataset['CLASSE']
X = dataset.iloc[:, :-1]

In [76]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo
...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,29/06/2018,fem div/cas,mobilia/equipamento
996,3857.0,4,40,nenhum,30/06/2018,masculino div/sep,carro usado
997,804.0,4,38,nenhum,03/07/2018,masculino solteiro,radio/tv
998,1845.0,4,23,nenhum,04/07/2019,masculino solteiro,radio/tv


### Valores Nulos

In [77]:
# verificando valores nulos
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [78]:
# saldo atual -> valor numérico: preenchendo com mediana
mediana = X['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [79]:
# preenchendo os valores de saldo atual
X['SALDO_ATUAL'].fillna(mediana, inplace=True)
X.isnull().sum()

C:\Users\Rogerio\AppData\Local\Temp\ipykernel_33976\2981516426.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana, inplace=True)


SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [80]:
# verificando a moda de estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [81]:
# preenchendo os valores de estado civil c/ moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

C:\Users\Rogerio\AppData\Local\Temp\ipykernel_33976\826143265.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)


SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

### Outliers

In [82]:
desvpad = X['SALDO_ATUAL'].std()
desvpad

np.float64(685936688.9820067)

In [83]:
# verificando onde há outliers
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [84]:
mediana = X['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [85]:
# eliminando os outliers (nesse caso, preenchendo com a mediana)
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad, 'SALDO_ATUAL'] = mediana
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


### Data Binding (Binning)

In [86]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [87]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

### Extração de Características

In [88]:
X['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [89]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [90]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [91]:
X['DIASEMANA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

### LabelEncoder

In [92]:
X['ESTADOCIVIL'].unique() # valores únicos

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [93]:
X['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [94]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [95]:
# Transformando Categorias
labelencoder1 = LabelEncoder()

X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [96]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [97]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [98]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


In [99]:
standscal = StandardScaler()
m = standscal.fit_transform(X.iloc[:, 0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]], shape=(1000, 3))

In [ ]:
X = pd.concat([X, z, pd.DataFrame(m, columns=['SALDO_ATUAL_N', 'RESIDENCIA_DESDE_N', 'IDADE_N'])], axis=1)
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIA_DESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,False,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0,False,True,False,-0.543562,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2,False,True,False,0.211898,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5,False,True,False,-0.875522,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4,False,True,False,-0.504738,1.046987,-0.687363


### Seleção de Características

In [102]:
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'])

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIA_DESDE_N,IDADE_N
0,3,7,2019,1,5,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,True,False,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,0,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566
997,3,7,2018,7,5,True,False,-0.875522,1.046987,0.105809
998,3,7,2019,7,4,True,False,-0.504738,1.046987,-0.687363
